164.431

vns
Selected Athletes:
['ou yushan' 'qiu qiyuan' 'tang xijing' 'zhou yaqin']
Total Score:
166.965

hh
Best Selection: ['bi qingqing' 'tang xijing' 'ou yushan' 'qiu qiyuan']
Best Score: 165.03199999999998

ts
['tang xijing' 'zhou yaqin' 'qiu qiyuan' 'ou yushan']
Total Score:
166.332

bf
Selected Athletes:
['bi qingqing', 'ou yushan', 'qiu qiyuan', 'tang xijing']
Maximum Score:
165.032

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [3]:
df = df[df['Nation']=='CHN']

In [4]:
df.duplicated().sum()

0

In [5]:
df=df.drop(columns=['Rank', 'Nation', 'AA', 'round', 'year'])

In [6]:
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values
})


In [10]:
df = df_max_scores.copy()

In [11]:
df

,Athlete,VT,UB,BB,FX
0,bi qingqing,12.950,0.000,13.250,13.100
1,chen shutong,12.850,12.000,11.750,12.400
2,he licheng,13.500,14.250,13.750,11.950
3,huang zhuofan,0.000,14.533,0.000,0.000
4,jia ruoyi,12.900,13.150,10.350,12.150
5,lin haibin,10.800,12.250,12.750,12.050
6,liu sibei,12.200,12.600,11.550,11.850
7,luo rui,0.000,14.900,10.766,0.000
8,ou yushan,13.100,14.300,14.066,13.400
9,qiu qiyuan,13.200,14.900,13.800,13.166


vns

In [12]:
import pandas as pd
import numpy as np


# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'UB', 'BB', 'VT']

# Define the neighborhood size
neighborhood_size = 2  # Adjust as needed

# Initialize the solution randomly
selected_athletes = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)

# Define a function to calculate the total score for a given solution
def calculate_total_score(selected_athletes):
    selected_scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].values
    top_3_scores = np.sum(np.partition(selected_scores, -3)[:, -3:])
    return top_3_scores

# Define the main VNS algorithm
def variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations):
    best_solution = None
    best_score = float('-inf')

    for iteration in range(num_iterations):
        # Shake: Generate a new solution by randomly changing athletes
        current_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
        
        for _ in range(neighborhood_size):
            # Local Search: Optimize the solution locally
            current_score = calculate_total_score(current_solution)
            
            # Perturb: Randomly change some athletes in the current solution
            perturbed_solution = np.random.choice(df['Athlete'], num_athletes_to_select, replace=False)
            perturbed_score = calculate_total_score(perturbed_solution)
            
            # Acceptance Criterion: Accept the perturbed solution if it improves the score
            if perturbed_score > current_score:
                current_solution = perturbed_solution
                current_score = perturbed_score

        # Update the best solution if the current one is better
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score

    return best_solution, best_score

# Run the VNS algorithm
best_solution, best_score = variable_neighborhood_search(df, num_athletes_to_select, apparatuses, neighborhood_size, num_iterations=1000)

# Print the result
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)


Selected Athletes:
['ou yushan' 'qiu qiyuan' 'tang xijing' 'zhou yaqin']
Total Score:
166.965


hh

In [13]:
import pandas as pd
import numpy as np



# Define the number of athletes to select
num_athletes_to_select = 4

# Define the apparatuses
apparatuses = ['FX', 'VT', 'BB', 'UB']

# Initialize variables to store the best solution
best_score = 0
best_selection = None

# Define the number of iterations for the hyper-heuristic
num_iterations = 1000

for _ in range(num_iterations):
    # Randomly select 4 athletes
    selected_athletes = np.random.choice(df['Athlete'], size=num_athletes_to_select, replace=False)

    # Calculate the score for the current selection
    scores = df[df['Athlete'].isin(selected_athletes)][apparatuses].sum(axis=1).nlargest(3).sum()

    # Update the best solution if the current one is better
    if scores > best_score:
        best_score = scores
        best_selection = selected_athletes

# Print the best selection and score
print("Best Selection:", best_selection)
print("Best Score:", best_score)


Best Selection: ['bi qingqing' 'tang xijing' 'ou yushan' 'qiu qiyuan']
Best Score: 165.03199999999998


ts


In [14]:
import pandas as pd
import numpy as np

def calculate_score(selected_athletes, df):
    """
    Calculate the total score for a given set of selected athletes.
    """
    # Select rows corresponding to the chosen athletes
    selected_df = df[df['Athlete'].isin(selected_athletes)]

    # Calculate the sum of the top 3 scores for each apparatus
    apparatus_scores = selected_df.drop(['Athlete'], axis=1)
    top_3_sum = apparatus_scores.apply(lambda row: sum(sorted(row, reverse=True)[:3]), axis=0)

    # Calculate the total score
    total_score = top_3_sum.sum()

    return total_score




def tabu_search(df, num_athletes_to_select, tabu_size, max_iterations):
    # Initialize the solution with a random set of athletes
    available_athletes = df['Athlete'].tolist()
    current_solution = np.random.choice(available_athletes, size=num_athletes_to_select, replace=False)
    best_solution = current_solution.copy()
    best_score = calculate_score(best_solution, df)

    # Initialize tabu list
    tabu_list = []

    # Main Tabu Search loop
    for iteration in range(max_iterations):
        # Generate neighbors by swapping athletes
        neighbors = []
        for i in range(num_athletes_to_select):
            for athlete in df['Athlete']:
                if athlete not in current_solution:
                    neighbor = current_solution.copy()
                    neighbor[i] = athlete
                    neighbors.append(neighbor)

                    
                    
                best_neighbor = None
        best_neighbor_score = float('-inf')

        for neighbor in neighbors:
            neighbor_score = calculate_score(neighbor, df)

            # Convert the NumPy array to a tuple
            neighbor_tuple = tuple(neighbor)

            if neighbor_score > best_neighbor_score and neighbor_tuple not in tabu_list:
                best_neighbor = neighbor
                best_neighbor_score = neighbor_score

        # Update current solution and tabu list
        current_solution = best_neighbor
        tabu_list.append(tuple(best_neighbor))

        # Keep the tabu list size within the specified limit
        if len(tabu_list) > tabu_size:
            tabu_list.pop(0)

        # Update the best solution if a better one is found
        if best_neighbor_score > best_score:
            best_solution = best_neighbor
            best_score = best_neighbor_score

        # Print progress
        print(f"Iteration {iteration + 1}, Best Score: {best_score}")

    return best_solution, best_score


# Set parameters
num_athletes_to_select = 4
tabu_size = 10
max_iterations = 100

# Perform Tabu Search
best_solution, best_score = tabu_search(df, num_athletes_to_select, tabu_size, max_iterations)

# Print the selected athletes and the corresponding score
print('Selected Athletes:')
print(best_solution)
print('Total Score:')
print(best_score)

      

Iteration 1, Best Score: 158.05
Iteration 2, Best Score: 163.666
Iteration 3, Best Score: 166.265
Iteration 4, Best Score: 166.332
Iteration 5, Best Score: 166.332
Iteration 6, Best Score: 166.332
Iteration 7, Best Score: 166.332
Iteration 8, Best Score: 166.332
Iteration 9, Best Score: 166.332
Iteration 10, Best Score: 166.332
Iteration 11, Best Score: 166.332
Iteration 12, Best Score: 166.332
Iteration 13, Best Score: 166.332
Iteration 14, Best Score: 166.332
Iteration 15, Best Score: 166.332
Iteration 16, Best Score: 166.332
Iteration 17, Best Score: 166.332
Iteration 18, Best Score: 166.332
Iteration 19, Best Score: 166.332
Iteration 20, Best Score: 166.332
Iteration 21, Best Score: 166.332
Iteration 22, Best Score: 166.332
Iteration 23, Best Score: 166.332
Iteration 24, Best Score: 166.332
Iteration 25, Best Score: 166.332
Iteration 26, Best Score: 166.332
Iteration 27, Best Score: 166.332
Iteration 28, Best Score: 166.332
Iteration 29, Best Score: 166.332
Iteration 30, Best Score

bf

In [15]:
import pandas as pd
import itertools
from tqdm import tqdm

num_athletes = 4
apparatuses = ['VT', 'UB', 'BB', 'FX']

# Generate all combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# variables to store maximum score and athletes
max_score = 0
selected_athletes = []

# tqdm progress bar
progress_bar = tqdm(total=len(athlete_combinations))

# Iterate through combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for current athlete combination
    apparatus_combinations = list(itertools.permutations(apparatuses, num_athletes))

    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate the sum of top three scores for current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()

        # Update maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

    # Update progress bar
    progress_bar.update(1)

progress_bar.close()

print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)


100%|█████████████████████████████████████| 14950/14950 [16:03<00:00, 15.51it/s]

Selected Athletes:
['bi qingqing', 'ou yushan', 'qiu qiyuan', 'tang xijing']
Maximum Score:
165.032


In [16]:

data = {
    'Algorithm': ['vns', 'hh', 'ts', 'bf'],
    'Athletes': [
        ['ou yushan', 'qiu qiyuan', 'tang xijing', 'zhou yaqin'],
        ['bi qingqing', 'tang xijing', 'ou yushan', 'qiu qiyuan'],
        ['tang xijing', 'zhou yaqin', 'qiu qiyuan', 'ou yushan'],
        ['bi qingqing', 'ou yushan', 'qiu qiyuan', 'tang xijing']
    ],
    'Score': [166.965, 165.032, 166.332, 165.032]
}

# Create DataFrame
chn = pd.DataFrame(data)

chn


,Algorithm,Athletes,Score
0,vns,"[ou yushan, qiu qiyuan, tang xijing, zhou yaqin]",166.965
1,hh,"[bi qingqing, tang xijing, ou yushan, qiu qiyuan]",165.032
2,ts,"[tang xijing, zhou yaqin, qiu qiyuan, ou yushan]",166.332
3,bf,"[bi qingqing, ou yushan, qiu qiyuan, tang xijing]",165.032


In [17]:
chn['nation']='CHN'

In [18]:
chn

,Algorithm,Athletes,Score,nation
0,vns,"[ou yushan, qiu qiyuan, tang xijing, zhou yaqin]",166.965,CHN
1,hh,"[bi qingqing, tang xijing, ou yushan, qiu qiyuan]",165.032,CHN
2,ts,"[tang xijing, zhou yaqin, qiu qiyuan, ou yushan]",166.332,CHN
3,bf,"[bi qingqing, ou yushan, qiu qiyuan, tang xijing]",165.032,CHN


In [19]:
chn.to_csv('china.csv', index=False)

In [7]:
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')


In [8]:
df = df[df['Nation']=='CHN']

In [11]:
df = df.drop(columns=['Nation', 'round', 'year', 'AA'])


In [12]:
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values
})

In [13]:
df=df_max_scores.copy()

In [14]:
df['AA'] = df.select_dtypes(include='number').sum(axis=1)

In [15]:
df=df.sort_values(by='AA', ascending=False)

In [16]:
df

,Athlete,VT,UB,BB,FX,AA
10,tang xijing,13.000,14.400,14.000,13.700,55.100
9,qiu qiyuan,13.200,14.900,13.800,13.166,55.066
8,ou yushan,13.100,14.300,14.066,13.400,54.866
13,wei xiaoyuan,12.950,14.900,13.300,12.733,53.883
21,zhang xinyi,13.000,13.750,14.050,12.800,53.600
15,xiang lulu,12.750,14.050,13.750,13.000,53.550
2,he licheng,13.500,14.250,13.750,11.950,53.450
17,zhang jin,13.833,12.250,13.850,12.700,52.633
18,zhang qingying,13.266,11.750,14.100,12.900,52.016
16,yue yue,12.950,13.900,12.400,12.400,51.650


In [17]:
55.1+55.066+54.866

165.03199999999998

In [2]:
df = pd.read_csv('../../Combine_Data/women/iaa2024.csv')
df = df.drop_duplicates()
df=df[df['Nation']=='CHN']
df=df.drop(columns=['Rank', 'Nation', 'round', 'year'])
# Group by 'Athlete' and get the index of the maximum score for each apparatus
idx_max = df.groupby('Athlete').idxmax()

# Create a new DataFrame with the highest scores for each athlete
df_max_scores = pd.DataFrame({
    'Athlete': df.loc[idx_max['VT'], 'Athlete'].values,
    'VT': df.loc[idx_max['VT'], 'VT'].values,
    'UB': df.loc[idx_max['UB'], 'UB'].values,
    'BB': df.loc[idx_max['BB'], 'BB'].values,
    'FX': df.loc[idx_max['FX'], 'FX'].values,
    'AA': df.loc[idx_max['AA'], 'AA'].values

})

In [3]:
df=df_max_scores.copy()

In [4]:
df.sort_values(by='AA', ascending=False)

,Athlete,VT,UB,BB,FX,AA
10,tang xijing,13.000,14.400,14.000,13.700,55.000
9,qiu qiyuan,13.200,14.900,13.800,13.166,54.799
8,ou yushan,13.100,14.300,14.066,13.400,54.632
13,wei xiaoyuan,12.950,14.900,13.300,12.733,53.700
21,zhang xinyi,13.000,13.750,14.050,12.800,53.600
15,xiang lulu,12.750,14.050,13.750,13.000,53.550
2,he licheng,13.500,14.250,13.750,11.950,53.450
17,zhang jin,13.833,12.250,13.850,12.700,52.500
16,yue yue,12.950,13.900,12.400,12.400,51.650
18,zhang qingying,13.266,11.750,14.100,12.900,51.250


In [5]:
55.000+54.799+54.632

164.431